In [ ]:
#Server prg
import socket,math,datetime,random,sys,os
import numpy as np
import pandas as pd
from scipy import constants
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
#import plotly.express as px
#import altair as alt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def is_float(a_string):
    try:
        float(a_string)
        return True
    except ValueError:
        return False
try:
    s=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((socket.gethostname() ,1234))  
    print(socket.gethostname())
    s.listen(5)
    while True:
        clt,adr=s.accept()
        msg=clt.recv(64000)
        xcode=msg.decode("utf-8")
        #print(xcode)
        if(xcode=="exit()" or xcode=="quit()"):
            print("output-0: ",xcode)
            sys.exit("You have stoped") 
            clt.close()
        elif(xcode.endswith(".csv")):
            df = pd.read_csv(xcode)
            print("output-00: ",xcode," loaded")
            clt.send("pandas dataframe object with 'df' loaded.".encode())
        elif(xcode.startswith("chart")):
            #if os.path.exists("plot.jpg"):
            #    os.remove("plot.jpg")
            #else:
            #    print("The file does not exist")
               
            plt.savefig("plot.jpg") # format='svg' dpi=300)   
            plt.close()
            with open('plot.jpg', 'rb') as file:
                image_data = file.read()
                clt.sendall(image_data)
        else:
            exec(f"out= "+xcode) #, globlsparam,  localsparam)
            output=str(out)
            
            if(output.isnumeric()):
                clt.send(bytes(out))
                print("output-1: ",out)
            elif (is_float(output)):
                clt.send(output.encode())
                print("output-1: ",output)
            else:
                clt.send(output.encode())
                print("output-2: ",output)
except Exception as err:
    print(f"Unexpected {err=}")
    error=str('Runtime Error: ') + str(err)
    clt.send(error.encode())
    clt.close()
    raise    